# Cooper 142 SNPs set


## Preparation

### Import required packages.

In [1]:
import os, sys, warnings
import numpy as np
import pandas as pd
import statistics as st
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.exceptions import ConvergenceWarning

### Read input matrix with genotypes

The matrix contains the genotypes from AMP-PD/MGRB dataset for 140 SNPs.

In [2]:
table = pd.read_csv("data/matrix.txt", sep="\t")
table

,participant_id,phenotype,cohort,gender,inv_genotype,rs2275579,rs144115304,rs115581042,rs79531911,rs138844738,...,rs10448130,rs34288580,rs34992950,rs7387252,rs2410595,rs41311559,rs148894916,rs112957100,rs143756122,rs148514732
0,SY-NIH_INVAA791MKCET,1,STEADY-PD3,M,NI,0,0,0,0,0,...,2,2,1,2,1,0,0,0,0,0
1,SY-NIH_INVEP886EEYYL,1,STEADY-PD3,M,NI,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,SY-NIH_INVFM717GWDX4,1,STEADY-PD3,F,NI,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,SY-NIH_INVNN611MKKN9,1,STEADY-PD3,M,NI,0,0,0,0,0,...,1,0,1,1,0,0,0,0,0,0
4,SY-NIH_INVRB171EXGUK,1,STEADY-PD3,M,II,0,1,1,1,0,...,1,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,BABQX,0,MGRB,M,II,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3108,BABRB,0,MGRB,F,II,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3109,BABRE,0,MGRB,M,NI,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3110,ZAAAB,0,MGRB,M,NI,0,0,0,0,0,...,2,1,1,1,0,0,0,0,0,0


### Distribution of data

#### Distribution by phenotype

(0=Control, 1=Case)

In [3]:
table.groupby('phenotype')['participant_id'].nunique()

phenotype
0    1556
1    1556
Name: participant_id, dtype: int64

#### Distribution by gender/phenotype

In [4]:
table.groupby(['gender', 'phenotype'])['participant_id'].nunique()

gender  phenotype
F       0            567
        1            567
M       0            989
        1            989
Name: participant_id, dtype: int64

#### Distribution by gender/phenotype/inv8_001 genotype

In [5]:
table.groupby(['gender', 'phenotype', 'inv_genotype'])['participant_id'].nunique()

gender  phenotype  inv_genotype
F       0          II              195
                   NI              259
                   NN              113
        1          II              175
                   NI              270
                   NN              122
M       0          II              318
                   NI              480
                   NN              191
        1          II              296
                   NI              477
                   NN              216
Name: participant_id, dtype: int64

## All participants

### Logistic regression model

In [6]:
pd.set_option('display.max_rows', 150)
X = table[table.columns[5:]]
Y = table['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table.groupby('phenotype')['participant_id'].nunique()

phenotype
0    1556
1    1556
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [7]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#              'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#              'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.01, 0.02, 0.05],
              'max_iter': [10, 25, 50],
              'l1_ratio': [1, 0.9, 0.8]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.02, 0.05], 'l1_ratio': [1, 0.9, 0.8],
                         'max_iter': [10, 25, 50]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [8]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best AUC score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.5552

Non-zero coefficients: 2

Best estimator: LogisticRegression(C=0.02, l1_ratio=1, max_iter=25, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.02, 'l1_ratio': 1, 'max_iter': 25}
Best AUC score: 0.5492



,SNP,Coefficient
Index,,
1,rs11248057,0.055163
2,rs3806760,0.098122


### 10-fold cross validation (5-times)

In [9]:
name = "all"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.5552
MeanCV AUC: 0.5512
Standard Deviation CV AUC: 0.0303


## Males

### Logistic regression model

In [10]:
table1 = table[table.gender == "M"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    989
1    989
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [11]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.01, 0.02],
              'max_iter': [75, 100, 150],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.02], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [75, 100, 150]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [12]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.6243

Non-zero coefficients: 37

Best estimator: LogisticRegression(C=0.01, l1_ratio=0.1, n_jobs=-1, penalty='elasticnet',
                   random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.01, 'l1_ratio': 0.1, 'max_iter': 100}
Best score: 0.5621



,SNP,Coefficient
Index,,
1,rs7520918,-0.10687
2,rs112270735,-0.077184
3,rs444618,-0.054004
4,rs13161496,-0.044109
5,rs3733349,-0.035941
6,rs3785891,-0.022054
7,rs494312,-0.015573
8,rs62190394,-0.008948
9,rs12995314,-0.00536


### 10-fold cross validation (5-times)

In [13]:
name = "M"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.6243
MeanCV AUC: 0.5682
Standard Deviation CV AUC: 0.0421


## Females

### Logistic regression model

In [14]:
table1 = table[table.gender == "F"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    567
1    567
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [15]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#              'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#              'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01],
              'max_iter': [25, 50, 75],
              'l1_ratio': [0.1, 0.2]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01], 'l1_ratio': [0.1, 0.2],
                         'max_iter': [25, 50, 75]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [16]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.5676

Non-zero coefficients: 6

Best estimator: LogisticRegression(C=0.005, l1_ratio=0.1, max_iter=50, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.005, 'l1_ratio': 0.1, 'max_iter': 50}
Best score: 0.5116



,SNP,Coefficient
Index,,
1,rs6871718,-0.026889
2,rs1949362,-0.023602
3,rs6964,0.005616
4,rs11248057,0.005966
5,rs3850379,0.008878
6,rs3806760,0.016905


### 10-fold cross validation (5-times)

In [17]:
name = "F"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.5676
MeanCV AUC: 0.5178
Standard Deviation CV AUC: 0.0490


## NN

### Logistic regression model

In [18]:
table1 = table[table.inv_genotype=="NN"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    304
1    338
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [19]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#              'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#              'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.01, 0.02, 0.05],
              'max_iter': [10, 25],
              'l1_ratio': [0.5, 0.4, 0.3]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.02, 0.05], 'l1_ratio': [0.5, 0.4, 0.3],
                         'max_iter': [10, 25]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [20]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.6094

Non-zero coefficients: 9

Best estimator: LogisticRegression(C=0.02, l1_ratio=0.3, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.02, 'l1_ratio': 0.3, 'max_iter': 10}
Best score: 0.5289



,SNP,Coefficient
Index,,
1,rs7826007,-0.013424
2,rs4897753,0.001234
3,rs3822023,0.005838
4,rs6842271,0.010758
5,rs12434297,0.012576
6,rs6964,0.014571
7,rs9985581,0.020144
8,rs3806760,0.031815
9,rs11248057,0.106701


### 10-fold cross validation (5-times)

In [21]:
name = "NN"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.6094
MeanCV AUC: 0.5483
Standard Deviation CV AUC: 0.0696


## NI

### Logistic regression model

In [22]:
table1 = table[table.inv_genotype=="NI"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    739
1    747
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [23]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01],
              'max_iter': [10, 25, 50],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [10, 25, 50]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [24]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.5797

Non-zero coefficients: 8

Best estimator: LogisticRegression(C=0.01, l1_ratio=0.2, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.01, 'l1_ratio': 0.2, 'max_iter': 10}
Best score: 0.5327



,SNP,Coefficient
Index,,
1,rs7520918,-0.045971
2,rs1949362,-0.017297
3,rs6532190,0.008321
4,rs17016235,0.00924
5,rs6842271,0.023003
6,rs9985581,0.03514
7,rs7682766,0.036363
8,rs3806760,0.047802


### 10-fold cross validation (5-times)

In [25]:
name = "NI"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.5797
MeanCV AUC: 0.5305
Standard Deviation CV AUC: 0.0400


## II

### Logistic regression model

In [26]:
table1 = table[table.inv_genotype=="II"]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    513
1    471
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [27]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01, 0.02],
              'max_iter': [50, 100, 200],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01, 0.02], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [50, 100, 200]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [28]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.6604

Non-zero coefficients: 26

Best estimator: LogisticRegression(C=0.01, l1_ratio=0.1, n_jobs=-1, penalty='elasticnet',
                   random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.01, 'l1_ratio': 0.1, 'max_iter': 100}
Best score: 0.5940



,SNP,Coefficient
Index,,
1,rs34333,-0.057329
2,rs494312,-0.05673
3,rs3733349,-0.055571
4,rs6871718,-0.051304
5,rs444618,-0.032258
6,rs1736103,-0.031732
7,rs7535292,-0.026481
8,rs13161496,-0.016119
9,rs4331494,-0.004139


### 10-fold cross validation (5-times)

In [29]:
name = "II"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.6604
MeanCV AUC: 0.5893
Standard Deviation CV AUC: 0.0544


## NN Males

### Logistic regression model

In [30]:
table1 = table[(table.gender == "M") & (table.inv_genotype=="NN")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    191
1    216
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [31]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.05, 0.1, 0.5],
              'max_iter': [10, 25],
              'l1_ratio': [0.7, 0.6, 0.5]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.05, 0.1, 0.5], 'l1_ratio': [0.7, 0.6, 0.5],
                         'max_iter': [10, 25]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [32]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.7224

Non-zero coefficients: 32

Best estimator: LogisticRegression(C=0.1, l1_ratio=0.6, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.1, 'l1_ratio': 0.6, 'max_iter': 10}
Best score: 0.5580



,SNP,Coefficient
Index,,
1,rs35776335,-0.213769
2,rs13161496,-0.193039
3,rs12499663,-0.150666
4,rs444618,-0.124434
5,rs112270735,-0.083023
6,rs1269243287,-0.055458
7,rs3912643,-0.048141
8,rs11097297,-0.047875
9,rs2128786,-0.03413


### 10-fold cross validation (5-times)

In [33]:
name = "M-NN"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.7224
MeanCV AUC: 0.5741
Standard Deviation CV AUC: 0.0898


## NI Males

### Logistic regression model

In [34]:
table1 = table[(table.gender == "M") & (table.inv_genotype=="NI")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    480
1    477
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [35]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01],
              'max_iter': [10, 25, 50],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [10, 25, 50]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [36]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.5836

Non-zero coefficients: 4

Best estimator: LogisticRegression(C=0.005, l1_ratio=0.1, max_iter=25, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.005, 'l1_ratio': 0.1, 'max_iter': 25}
Best score: 0.5213



,SNP,Coefficient
Index,,
1,rs7520918,-0.011899
2,rs9985581,0.016555
3,rs6842271,0.016555
4,rs3806760,0.027075


### 10-fold cross validation (5-times)

In [37]:
name = "M-NI"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.5836
MeanCV AUC: 0.5271
Standard Deviation CV AUC: 0.0600


## II Males

### Logistic regression model

In [38]:
table1 = table[(table.gender == "M") & (table.inv_genotype=="II")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    318
1    296
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [39]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.01, 0.02],
              'max_iter': [50, 75, 100],
              'l1_ratio': [0.2, 0.1]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.02], 'l1_ratio': [0.2, 0.1],
                         'max_iter': [50, 75, 100]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [40]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.6759

Non-zero coefficients: 18

Best estimator: LogisticRegression(C=0.01, l1_ratio=0.1, max_iter=75, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.01, 'l1_ratio': 0.1, 'max_iter': 75}
Best score: 0.6007



,SNP,Coefficient
Index,,
1,rs34333,-0.092096
2,rs3733349,-0.053093
3,rs3785891,-0.013793
4,rs8064765,-0.003056
5,rs62075803,-0.002828
6,rs4796663,-0.002828
7,rs4028634,0.001729
8,rs2410595,0.003947
9,rs764324,0.00623


### 10-fold cross validation (5-times)

In [41]:
name = "M-II"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.6759
MeanCV AUC: 0.6040
Standard Deviation CV AUC: 0.0706


## NN Females

### Logistic regression model

In [42]:
table1 = table[(table.gender == "F") & (table.inv_genotype=="NN")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    113
1    122
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [43]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.1, 0.5, 1],
              'max_iter': [10, 25],
              'l1_ratio': [0.7, 0.6, 0.5]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.1, 0.5, 1], 'l1_ratio': [0.7, 0.6, 0.5],
                         'max_iter': [10, 25]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [44]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.8582

Non-zero coefficients: 112

Best estimator: LogisticRegression(C=0.5, l1_ratio=0.6, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.5, 'l1_ratio': 0.6, 'max_iter': 10}
Best score: 0.5680



,SNP,Coefficient
Index,,
1,rs6871718,-0.426813
2,rs999361,-0.426221
3,rs7826007,-0.365302
4,rs3733349,-0.346928
5,rs78197677,-0.284285
6,rs7535292,-0.26294
7,rs4859611,-0.257429
8,rs1949362,-0.254682
9,rs11601088,-0.247447


### 10-fold cross validation (5-times)

In [45]:
name = "F-NN"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.8582
MeanCV AUC: 0.5360
Standard Deviation CV AUC: 0.1143


## NI Females

### Logistic regression model

In [46]:
table1 = table[(table.gender == "F") & (table.inv_genotype=="NI")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    259
1    270
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [47]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#              'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#              'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [0.005, 0.01],
              'max_iter': [10, 25],
              'l1_ratio': [1, 0.9]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.005, 0.01], 'l1_ratio': [1, 0.9],
                         'max_iter': [10, 25]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [48]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.5000

Non-zero coefficients: 0

Best estimator: LogisticRegression(C=0.005, l1_ratio=1, max_iter=10, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 0.005, 'l1_ratio': 1, 'max_iter': 10}
Best score: 0.5000



,SNP,Coefficient
Index,,


### 10-fold cross validation (5-times)

In [49]:
name = "F-NI"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.5000
MeanCV AUC: 0.5000
Standard Deviation CV AUC: 0.0000


## II Females

### Logistic regression model

In [50]:
table1 = table[(table.gender == "F") & (table.inv_genotype=="II")]
X = table1[table1.columns[5:]]
Y = table1['phenotype']
lr = LogisticRegression(random_state=42, solver='saga', n_jobs=-1, penalty='elasticnet')
table1.groupby('phenotype')['participant_id'].nunique()

phenotype
0    195
1    175
Name: participant_id, dtype: int64

### Grid search for 3 hyperparameters

In [51]:
# parameters = {'C': [0.005, 0.01, 0.02, 0.05, 0.1, 0.5, 1, 10, 20, 30],
#               'max_iter': [10, 25, 50, 75, 100, 150, 200, 400, 800, 1600],
#               'l1_ratio': [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]}

parameters = {'C': [1, 10, 20],
              'max_iter': [1600, 3200],
              'l1_ratio': [1, 0.9]}

grid_lr = GridSearchCV(lr, parameters, verbose=False, scoring='roc_auc', n_jobs=-1, cv=10)
if not sys.warnoptions:
    warnings.simplefilter("ignore", category=ConvergenceWarning)
    os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses
grid_lr.fit(X, Y)

GridSearchCV(cv=10,
             estimator=LogisticRegression(n_jobs=-1, penalty='elasticnet',
                                          random_state=42, solver='saga'),
             n_jobs=-1,
             param_grid={'C': [1, 10, 20], 'l1_ratio': [1, 0.9],
                         'max_iter': [1600, 3200]},
             scoring='roc_auc', verbose=False)

### Best estimator

In [52]:
best_lr = grid_lr.best_estimator_

max_auc_score = roc_auc_score(Y, best_lr.predict_proba(X)[:, 1])
coefs = best_lr.coef_[0, :]
num_coef = np.sum(coefs != 0)
X_header = np.array(X.columns)

data_array = np.vstack((X_header, coefs))
model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
print(f'Max AUC score:{max_auc_score:.4f}\n')
print(f'Non-zero coefficients: {num_coef}\n')
print(f'Best estimator: {grid_lr.best_estimator_}')
print(f'Scorer: {grid_lr.scorer_}')
print(f'Best params: {grid_lr.best_params_}')
print(f'Best score: {grid_lr.best_score_:.4f}\n')
m = model_coefs[model_coefs['Coefficient'] != 0 ].sort_values(by='Coefficient')
m = m.reset_index(drop=True).assign(Index=range(len(m)))
m.Index= m.Index + 1
m.set_index('Index')

Max AUC score:0.8848

Non-zero coefficients: 131

Best estimator: LogisticRegression(C=10, l1_ratio=1, max_iter=3200, n_jobs=-1,
                   penalty='elasticnet', random_state=42, solver='saga')
Scorer: make_scorer(roc_auc_score, needs_threshold=True)
Best params: {'C': 10, 'l1_ratio': 1, 'max_iter': 3200}
Best score: 0.5429



,SNP,Coefficient
Index,,
1,rs138844738,-4.155261
2,rs115879964,-4.155261
3,rs536718528,-3.617586
4,rs115448944,-3.150895
5,rs71607338,-2.57904
6,rs17324625,-2.427067
7,rs2433733,-1.988805
8,rs6842271,-1.715214
9,rs62075803,-1.652955


### 10-fold cross validation (5-times)

In [53]:
name = "F-II"
AUCs = list()
bp = grid_lr.best_params_
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = LogisticRegression(C=bp['C'], max_iter=bp['max_iter'], l1_ratio=bp['l1_ratio'], random_state=42,
      solver='saga', n_jobs=-1, penalty='elasticnet')

count = 0
for train_index, test_index in rkf.split(X):
    count = count + 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    clf.fit(X_train, y_train)

    y_pred = clf.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, y_pred)
    AUCs.append(auc)

    X_header = np.array(X_train.columns)
    data_array = np.vstack((X_header, clf.coef_[0,:]))
    model_coefs = pd.DataFrame(data=data_array.T, columns=['SNP', 'Coefficient'])
    m_name = f'data/{name}_10fold_repeat{count}_coefficients.txt'
    model_coefs.to_csv(m_name, sep='\t',index=False)

# Fit predictor to statistically significant features (just once!!!)
clf.fit(X, Y)
y_pred = clf.predict_proba(X)[:,1]

# This in-sample AUC should be better than the AUCs from the repeated cross-validation
auc = roc_auc_score(Y, y_pred)

#AUC results from the 50 predictors
AUC_out = pd.DataFrame(AUCs, columns=['AUC'])
AUC_out.to_csv(f"data/{name}_AUCs.txt", sep='\t',index=False)

AUC_std= st.stdev(AUCs)
AUC_mean= st.mean(AUCs)

print(f'In-Sample AUC: {auc:.4f}')
print(f'MeanCV AUC: {AUC_mean:.4f}')
print(f'Standard Deviation CV AUC: {AUC_std:.4f}')

In-Sample AUC: 0.8848
MeanCV AUC: 0.5567
Standard Deviation CV AUC: 0.0876
